In [ ]:
import datetime
import os
import json
from datetime import timezone
from tzlocal import get_localzone
from typing import TypedDict, Annotated, List, Optional
import operator

from dotenv import load_dotenv;
from pydantic import  BaseModel, Field

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import base64

from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.sqlite import SqliteSaver


from IPython.display import Image, display
from dateutil.parser import parse, ParserError

In [ ]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY") 

SCOPES = ["https://www.googleapis.com/auth/calendar", "https://www.googleapis.com/auth/gmail.readonly"]

if not all([groq_api_key, tavily_api_key]):
    raise ValueError("One or more required API keys (GROQ, TAVILY) are missing from the .env file!")


llm = ChatGroq(model="llama3-70b-8192", api_key=groq_api_key, temperature=0)
print("Groq LLM (Llama3-70b) configured and ready.")

In [ ]:
class CalendarEvent(BaseModel):
    """Schema for a single, richly detailed calendar event."""
    id: str = Field(description="The unique ID of the calendar event.")
    summary: str = Field(description="The title or summary of the calendar event.")
    start_time: str = Field(description="The start time of the event in ISO format.")
    source: str = Field(description="The source of the calendar, either 'Google' or 'Apple'.")
    description: Optional[str] = Field(default=None, description="The detailed description or notes for the event.")
    attendees: Optional[List[str]] = Field(default=None, description="A list of email addresses of the event attendees.")
    location: Optional[str] = Field(default=None, description="The physical location of the event (e.g., an address or room name).")
    video_conference_link: Optional[str] = Field(default=None, description="The link for the Google Meet video conference, if available.")
    attachments: Optional[List[str]] = Field(default=None, description="A list of Google Drive attachments, formatted as 'title (link)'.")



def _get_google_credentials():
    """Gets valid Google API credentials."""
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    return creds



def _fetch_google_events(start_date: datetime.datetime, end_date: datetime.datetime) -> List[dict]:
    """
    Internal function to fetch events from Google Calendar, now including ALL rich details:
    attendees, description, location, conference links, and attachments.
    """
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)
        
        events_result = service.events().list(
            calendarId="primary",
            timeMin=start_date.isoformat(),
            timeMax=end_date.isoformat(),
            maxResults=250,
            singleEvents=True,
            orderBy="startTime"
        ).execute()
        
        events = events_result.get("items", [])
        if not events: return []

        output_events = []
        for event in events:
            start = event["start"].get("dateTime", event["start"].get("date"))
            
            description = event.get("description")
            
            attendee_emails = [att.get("email") for att in event.get("attendees", []) if att.get("email")]

            location = event.get("location")
            
            video_conference_link = None
            if "conferenceData" in event and "entryPoints" in event["conferenceData"]:
                for entry_point in event["conferenceData"]["entryPoints"]:
                    if entry_point.get("entryPointType") == "video":
                        video_conference_link = entry_point.get("uri")
                        break 
            
            attachment_info = []
            if "attachments" in event:
                for attachment in event["attachments"]:
                    title = attachment.get("title", "Untitled Attachment")
                    link = attachment.get("fileUrl", "#")
                    attachment_info.append(f"{title} ({link})")

            event_model = CalendarEvent(
                id=event.get("id"),
                summary=event.get("summary", "No Title"), 
                start_time=start, 
                source="Google",
                description=description,
                attendees=attendee_emails if attendee_emails else None,
                location=location,
                video_conference_link=video_conference_link,
                attachments=attachment_info if attachment_info else None 
            )
            output_events.append(event_model.model_dump())
        return output_events
    except Exception as e:
        print(f"!!! Google Calendar Read Error: {e}")
        return [{"error": f"An error occurred while reading Google Calendar: {e}"}]


In [ ]:
def _get_email_body(parts):
    """Recursively search for text/plain parts in email body."""
    body = ""
    if parts:
        for part in parts:
            if part['mimeType'] == 'text/plain' and 'data' in part['body']:
                body += base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            elif 'parts' in part:
                body += _get_email_body(part['parts'])
    return body

def _fetch_gmail_messages(query: str, max_results: int = 25) -> List[dict]:
    """Internal function to fetch and sort emails from Gmail using batch processing."""
    try:
        creds = _get_google_credentials()
        service = build("gmail", "v1", credentials=creds)

        message_list_request = service.users().messages().list(userId='me', q=query, maxResults=max_results)
        message_list = message_list_request.execute()
        
        messages = message_list.get('messages', [])
        if not messages:
            return []

        output_emails = []
        
        def gmail_callback(request_id, response, exception):
            if exception is not None:
                print(f"Error in batch request {request_id}: {exception}")
            else:
                headers = response['payload']['headers']
                subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'No Subject')
                sender = next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown Sender')
                date = next((h['value'] for h in headers if h['name'] == 'Date'), 'Unknown Date')
                snippet = response['snippet']
                internal_date = response['internalDate']
                
                body = ""
                if 'parts' in response['payload']:
                    body = _get_email_body(response['payload']['parts'])
                elif 'data' in response['payload']['body']:
                    body = base64.urlsafe_b64decode(response['payload']['body']['data']).decode('utf-8')

                output_emails.append({
                    "id": response['id'],
                    "sender": sender,
                    "subject": subject,
                    "date": date,
                    "snippet": snippet,
                    "body": body[:2000],
                    "internalDate": internal_date
                })

        batch_request = service.new_batch_http_request(callback=gmail_callback)

        for msg in messages:
            request = service.users().messages().get(userId='me', id=msg['id'], format='full')
            batch_request.add(request)

        print(f"--- Executing batch request for {len(messages)} emails ---")
        batch_request.execute()
        print(f"--- Batch request completed ---")

        if output_emails:
            output_emails.sort(key=lambda e: int(e['internalDate']), reverse=True)

        return output_emails

    except HttpError as error:
        print(f"!!! Gmail API Error: {error}")
        return [{"error": f"An error occurred with the Gmail API: {error}"}]
    except Exception as e:
        print(f"!!! An unexpected error occurred: {e}")
        return [{"error": f"An unexpected error occurred: {e}"}]

In [ ]:
@tool
def list_upcoming_events(limit: Optional[int] = 15) -> str:
    """
    Lists the user's next upcoming events from today onwards from Google Calendar.
    Use this tool for general queries like 'list my events' or 'what's next?'.
    This tool does NOT take a specific date parameter.
    """
    print(f"--- Tool: list_upcoming_events called with limit={limit} ---")
    
    now = datetime.datetime.now(timezone.utc)
    search_end = now + datetime.timedelta(days=90) 

    google_events = _fetch_google_events(start_date=now, end_date=search_end)
    
    if not google_events or all('error' in e for e in google_events):
        return json.dumps([{"message": "No upcoming events found."}])

    valid_events = [e for e in google_events if 'error' not in e]
    
    if not limit: limit = 15
    events_to_return = valid_events[:limit]
    
    return json.dumps(events_to_return, indent=2)


@tool
def get_events_for_date(target_date: str) -> str:
    """
    Gets all Google Calendar events for a specific given date, including rich
    details like the description, attendees, location, and conference links.
    Use this tool when the user asks about a particular date (e.g., 'tomorrow', 'August 22, 2025').
    This tool REQUIRES the 'target_date' parameter in 'YYYY-MM-DD' format.
    """
    print(f"--- Tool: get_events_for_date called with date='{target_date}' ---")
    
    try:
        parsed_target_date = parse(target_date).date()
    except ParserError:
        return json.dumps([{"error": f"Invalid date format provided: {target_date}"}])


    start_of_day = datetime.datetime.combine(parsed_target_date, datetime.time.min).replace(tzinfo=timezone.utc)
    end_of_day = datetime.datetime.combine(parsed_target_date, datetime.time.max).replace(tzinfo=timezone.utc)
    
    google_events = _fetch_google_events(start_date=start_of_day, end_date=end_of_day)
    valid_events = [e for e in google_events if 'error' not in e]
    
    if not valid_events:
        return json.dumps([])

    return json.dumps(valid_events, indent=2)


@tool
def tavily_search(query: str) -> str:
    """
    A search engine tool to find real-time information online.
    Use this for any questions about weather, news, current events, facts,
    or general knowledge that is not related to the user's personal calendar.
    Do NOT claim you cannot answer these types of questions.
    """
    print(f"--- Tool: tavily_search called with query: '{query}' ---")
    try:
        search = TavilySearch(max_results=3, api_key=tavily_api_key)
        results = search.invoke(query)
        
        if not results:
            return "No results found from web search."

        
        return results

        
        
    except Exception as e:
        print(f"!!! TAVILY SEARCH FAILED: {e}")
        return f"Error occurred during web search: {e}"



@tool
def create_google_event(
    summary: str, 
    start_time_iso: str, 
    end_time_iso: str, 
    description: Optional[str] = None, 
    location: Optional[str] = None, 
    attendees: Optional[List[str]] = None
) -> str:
    """
    Creates a new event on the user's primary Google Calendar after checking for conflicts.

    Args:
        summary (str): The title or summary of the event.
        start_time_iso (str): The start time in ISO 8601 format (e.g., '2025-08-18T15:00:00').
        end_time_iso (str): The end time in ISO 8601 format (e.g., '2025-08-18T16:00:00').
        description (Optional[str]): A detailed description or notes for the event.
        location (Optional[str]): The physical location or address of the event.
        attendees (Optional[List[str]]): A list of attendee email addresses to invite.
    """
    print(f"--- Tool: create_google_event called with: summary='{summary}', start='{start_time_iso}', end='{end_time_iso}', description='{description}', location='{location}', attendees='{attendees}' ---")
    
    try:

        start_time_obj = parse(start_time_iso)
        end_time_obj = parse(end_time_iso)
        
        local_tz = get_localzone()

        if start_time_obj.tzinfo is None:
            start_time_obj = start_time_obj.replace(tzinfo=local_tz)
        if end_time_obj.tzinfo is None:
            end_time_obj = end_time_obj.replace(tzinfo=local_tz)

        conflicting_events = _fetch_google_events(start_date=start_time_obj, end_date=end_time_obj)
        valid_conflicts = [e for e in conflicting_events if 'error' not in e]

        if valid_conflicts:
            conflict_summary = valid_conflicts[0].get('summary', 'an existing event')
            print(f"--- Failure: Event conflict found with '{conflict_summary}' ---")
            return f"Error: Cannot create event. There is a conflicting event at that time: '{conflict_summary}'."

        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)
        
        local_timezone = str(get_localzone())
        
        event_body = {
            'summary': summary,
            'start': {'dateTime': start_time_iso, 'timeZone': local_timezone},
            'end': {'dateTime': end_time_iso, 'timeZone': local_timezone},
        }

        if description:
            event_body['description'] = description
        
        if location:
            event_body['location'] = location
        
        if attendees:
            event_body['attendees'] = [{'email': email} for email in attendees]

        created_event = service.events().insert(calendarId='primary', body=event_body).execute()
        
        print(f"--- Success: Event created. ID: {created_event.get('id')} ---")
        return f"Event '{summary}' was created successfully with all provided details."

    except Exception as e:
        print(f"!!! Google Calendar Write Error: {e}")
        return f"An error occurred while creating the event in Google Calendar: {e}"



@tool
def delete_google_event(event_id: str, summary: str) -> str:
    """
    Deletes an event from the user's primary Google Calendar using its unique ID.

    To use this tool, you must first find the event and its 'id' by using
    `get_events_for_date` or `list_upcoming_events`.

    Args:
        event_id (str): The unique ID of the event to be deleted.
        summary (str): The summary/title of the event, used for the confirmation message.
    """
    print(f"--- Tool: delete_google_event called for event ID: {event_id} ---")
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)

        service.events().delete(
            calendarId='primary',
            eventId=event_id
        ).execute()
        
        print(f"--- Success: Event '{summary}' deleted. ---")
        return f"The event '{summary}' was successfully deleted from the calendar."

    except Exception as e:
        print(f"!!! Google Calendar Delete Error: {e}")
        return f"An error occurred while trying to delete the event: {e}"


@tool
def get_calendar_summary(time_range: str) -> str:
    """
    Provides a summary of Google Calendar events for a given time range.
    Use this for questions like "what's my schedule this week?" or "give me my weekly report".

    Args:
        time_range (str): The time range to summarize. Supported values: "today", "tomorrow", "this week", "next week".
    """
    print(f"--- Tool: get_calendar_summary called for range: {time_range} ---")
    today = datetime.date.today()
    
    if time_range == "today":
        start_date = today
        end_date = today
    elif time_range == "tomorrow":
        start_date = today + datetime.timedelta(days=1)
        end_date = start_date
    elif time_range == "this week":
        start_date = today - datetime.timedelta(days=today.weekday())
        end_date = start_date + datetime.timedelta(days=6)
    elif time_range == "next week":
        start_date = today - datetime.timedelta(days=today.weekday()) + datetime.timedelta(days=7)
        end_date = start_date + datetime.timedelta(days=6)
    else:
        return json.dumps([{"error": "Unsupported time range. Please use 'today', 'tomorrow', 'this week', or 'next week'."}])

    start_datetime = datetime.datetime.combine(start_date, datetime.time.min).replace(tzinfo=timezone.utc)
    end_datetime = datetime.datetime.combine(end_date, datetime.time.max).replace(tzinfo=timezone.utc)
    
    events = _fetch_google_events(start_date=start_datetime, end_date=end_datetime)
    
    return json.dumps(events, indent=2)



@tool
def update_google_event(
    event_id: str,
    summary: Optional[str] = None,
    start_time_iso: Optional[str] = None,
    end_time_iso: Optional[str] = None,
    description: Optional[str] = None,
    location: Optional[str] = None,
    attendees: Optional[List[str]] = None
) -> str:
    """
    Updates an existing event on the user's primary Google Calendar using its unique ID.
    To use this tool, you must first find the event and its 'id'.
    Only provide the fields that are being changed.
    If only start_time_iso is provided, the original duration will be maintained.
    """
    print(f"--- Tool: update_google_event called for event ID: {event_id} ---")
    try:
        creds = _get_google_credentials()
        service = build("calendar", "v3", credentials=creds)

        event = service.events().get(calendarId='primary', eventId=event_id).execute()

        if start_time_iso and not end_time_iso:
            start_dt_obj = parse(event["start"].get("dateTime"))
            end_dt_obj = parse(event["end"].get("dateTime"))
            original_duration = end_dt_obj - start_dt_obj
            
            new_start_dt_obj = parse(start_time_iso)
            new_end_dt_obj = new_start_dt_obj + original_duration
            end_time_iso = new_end_dt_obj.isoformat()

        update_body = {}
        local_timezone = str(get_localzone())

        if summary:
            update_body['summary'] = summary
        if description:
            update_body['description'] = description
        if location:
            update_body['location'] = location
        if attendees is not None: 
            update_body['attendees'] = [{'email': email} for email in attendees]
        if start_time_iso:
            update_body['start'] = {'dateTime': start_time_iso, 'timeZone': local_timezone}
        if end_time_iso:
            update_body['end'] = {'dateTime': end_time_iso, 'timeZone': local_timezone}

        if not update_body:
            return "Error: No update information was provided."

        updated_event = service.events().patch(
            calendarId='primary',
            eventId=event_id,
            body=update_body
        ).execute()
        
        final_summary = updated_event.get('summary')
        start = parse(updated_event["start"].get("dateTime")).strftime("%Y-%m-%d %H:%M")
        end = parse(updated_event["end"].get("dateTime")).strftime("%Y-%m-%d %H:%M")

        print(f"--- Success: Event '{final_summary}' updated. ---")
        return f"Success! Event '{final_summary}' was updated to take place from {start} to {end}."

    except HttpError as err:
        error_reason = err.reason if hasattr(err, 'reason') else 'Unknown API Error'
        print(f"!!! Google API HTTP Error: {err.resp.status} - {error_reason} !!!")
        if err.resp.status == 404:
            return f"Error: The event with ID '{event_id}' was not found. It may have been deleted."
        return f"An error occurred with the Google Calendar API: {error_reason}"
    except Exception as e:
        print(f"!!! Google Calendar Generic Update Error: {e}")
        return f"An unexpected error occurred while trying to update the event: {e}"


In [ ]:
@tool
def read_and_summarize_emails(
    sender: Optional[str] = None,
    subject: Optional[str] = None,
    keywords: Optional[str] = None,
    status: Optional[str] = "unread",
    time_range: Optional[str] = None,
    max_results: int = 25
) -> str:
    """
    Reads and summarizes emails from Gmail using advanced search queries.

    Args:
        sender (Optional[str]): Filter emails from a specific sender (e.g., 'Google').
        subject (Optional[str]): Filter emails that contain specific words in their subject line.
        keywords (Optional[str]): Free-form search for keywords anywhere in the email.
        status (Optional[str]): Filter by status. Accepts 'unread', 'read', or 'all'. Defaults to 'unread'.
        time_range (Optional[str]): Filter emails newer than a specific time. Examples: '2d' (2 days), '6h' (6 hours).
        max_results (int): The maximum number of emails to fetch. Defaults to 25.
    """

    print(f"--- Tool: read_and_summarize_emails called with sender='{sender}', subject='{subject}', keywords='{keywords}', status='{status}', time_range='{time_range}' ---")
    
    query_parts = []
    
    if status == "unread":
        query_parts.append("is:unread")
    elif status == "read":
        query_parts.append("is:read")

    if sender:
        query_parts.append(f"from:{sender}")

    if subject:
        query_parts.append(f"subject:({subject})")

    if keywords:
        query_parts.append(keywords)
    
    if time_range:
        query_parts.append(f"newer_than:{time_range}")
            
    query = " ".join(query_parts)
    
    print(f"--- Generated Gmail Query: q='{query}' ---")

    emails = _fetch_gmail_messages(query=query, max_results=max_results)

    if not emails or all('error' in e for e in emails):
        return "No emails found matching the criteria or an error occurred."

    if len(emails) == 0:
        return "No recent emails found matching your criteria."
    
    summaries = []
    for email in emails:
        summarization_prompt = f"""
        Please provide a concise, one-sentence summary of the following email.
        Focus on the main point and any call to action.

        From: {email['sender']}
        Subject: {email['subject']}
        Date: {email['date']} 
        Body Snippet: {email['body']}
        """
        summary_response = llm.invoke(summarization_prompt)
        
        summaries.append(f"-From: {email['sender']} (Received: {email['date']})\n  Subject: {email['subject']}\n  Summary: {summary_response.content}\n")
    
    return f"Here are the summaries of the most recent emails:\n\n" + "\n".join(summaries)

In [ ]:
tools = [
    list_upcoming_events,
    get_events_for_date,
    create_google_event,
    delete_google_event,
    get_calendar_summary,
    update_google_event,
    read_and_summarize_emails,
    tavily_search]
    
print(f"Defined {len(tools)} tools for the agent: {[t.name for t in tools]}")

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]


llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools)


system_prompt_template = """
You are a multi-talented personal assistant. Today's date is {current_date}.

**STEP 1: FIRST, DECIDE THE USER'S INTENT**
Before doing anything, first, determine if the user's message is 'Conversational' or a 'Task'.

-   **'Conversational' Intent:** Simple greetings, social chat, thank yous, or opinions.
    -   **Action:** You MUST respond conversationally WITHOUT using any tools.

-   **'Task' Intent:** Any request that requires information or action (calendar, email, web search).
    -   **Action:** If it is a 'Task', proceed to STEP 2.

**STEP 2: IF IT IS A 'TASK', EXECUTE IT USING THESE RULES**

1.  **TOOL-FIRST PRINCIPLE (FOR TASKS ONLY):** For ANY 'Task', you are FORBIDDEN from answering from memory. You MUST use a tool. If a tool returns "No events found", inform the user and stop. Do not try another tool.

2.  **TOOL SELECTION LOGIC:**
    -   For **SPECIFIC DATE** calendar questions, use `get_events_for_date`.
    -   For **GENERAL** calendar questions, use `list_upcoming_events`.
    -   For **EMAIL** questions, use `read_and_summarize_emails` with the correct filters (sender, subject, etc.).
    -   For all other general knowledge questions, use `tavily_search`.

3.  **PROCEDURAL RULES FOR ACTIONS (MUST BE FOLLOWED EXACTLY, NO SKIPPING STEPS):**
    -   **Create Event:** You MUST have the title, start time, AND duration before calling `create_google_event`. If duration is missing, you MUST ask the user for it.
    -   **Update Event (STRICT 2-Step Process):**
        1.  **SEARCH:** When a user asks to modify an event, your FIRST AND ONLY initial action is to call `get_events_for_date` to find the event and get its real `id`. Do NOT proceed without a real `id` from this tool.
        2.  **EXECUTE:** After you have the real `id`, make a SECOND, separate call to `update_google_event` with that `id` and the new information.
    -   **Delete Event (STRICT 3-Step Process):**
        1.  **SEARCH:** First, your ONLY initial action is to call `get_events_for_date` to find the event and get its `id`.
        2.  **CONFIRM:** Second, after finding the event, you MUST ask the user for confirmation: "I found '[Event Name]'. Are you sure you want to delete it?". You MUST wait for their "yes" or "no".
        3.  **EXECUTE:** Third, ONLY AFTER the user explicitly confirms, make a FINAL call to `delete_google_event` with the `id`.
"""

def agent_node(state):
    today_str = datetime.date.today().isoformat()
    formatted_prompt = system_prompt_template.format(current_date=today_str)
    
    current_messages = state['messages']
    if not current_messages or not isinstance(current_messages[0], SystemMessage):
        messages_with_prompt = [SystemMessage(content=formatted_prompt)] + current_messages
    else:
        current_messages[0] = SystemMessage(content=formatted_prompt)
        messages_with_prompt = current_messages

    response = llm_with_tools.invoke(messages_with_prompt)
    return {"messages": [response]}


def should_continue(state):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")


In [ ]:
db_path = os.path.abspath("conversations.sqlite")

print(f"Using database at: {db_path}")

with SqliteSaver.from_conn_string(db_path) as memory:
    
    app = workflow.compile(checkpointer=memory)

    print("\nGraph is compiled with persistent memory. Ready for interaction.")

    config = {"configurable": {"thread_id": "berky_alkn22"}}

    print("\nYour persistent assistant is ready. You can now start chatting.")
    print("   Type 'quit' or 'exit' to end the conversation.")
    print("   To start a new conversation, restart the script and change the 'thread_id'.")
    print("-" * 50)

    while True:
       
        try:
            user_input = input("You: ")
            if user_input.lower() in ["quit", "exit"]:
                print("Assistant: Goodbye!")
                break

            print(f"You: {user_input}")

            response = app.invoke(
                {"messages": [HumanMessage(content=user_input)]},config
            )
        
            final_response = response["messages"][-1]
            print(f"Assistant: {final_response.content}")
        
        except Exception as e:
            print(f"\n--- An Error Occurred ---")
            print(f"Error: {e}")
            print("Please try again.")